In [1]:
"""
Summaries of USFS TreeMap linked to FIA plot data
Emphasis on 
    - Metrics of forest composition
    - Ecological gradients of species dominance
    - Forest structure (abundance, dominance, diversity, stand height)

Aggregate these statistics to FRP gridcells.

Author: maxwell.cook@colorado.edu
"""

import os, sys, time
import pandas as pd
import rioxarray as rxr
import xarray as xr
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import to_rgba

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

albers = 'EPSG:5070' # albers CONUS
utm = 'EPSG:32613' # UTM Zone 13N

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Ready to go !")

Ready to go !


In [19]:
# load the aggregated FRP grids (regular 375m2 grids summarizing FRP from VIIRS)
fp = os.path.join(projdir,'data/spatial/mod/VIIRS/viirs_snpp_jpss1_afd_latlon_fires_pixar_gridstats.gpkg')
grid = gpd.read_file(fp)
print(f"\nThere are [{len(grid)}] grids across [{len(grid['Fire_ID'].unique())}] fires.\n")

# add the centroid lat/lon to the grid data
df = grid.to_crs(4326) # WGS coords for lat/lon
df['x'] = df.geometry.centroid.x  # Longitude (x-coordinate)
df['y'] = df.geometry.centroid.y
grid = grid.merge(df[['grid_index','x','y']], on='grid_index', how='left')
del df
print(f"\n{grid.columns}\n")

# Drop any dupicate grids ...
print(f"Dropping [{grid.duplicated(subset=['grid_index']).sum()}] duplicate grids.\n")
grid = grid.drop_duplicates(subset=['grid_index'], keep='first')

# check on fractional overlap distribution
print(f"Fractional overlap:\n{grid['overlap'].describe()}\n")
n_small = grid[grid['overlap'] < 0.10]['grid_index'].count() # less than 10% spatial overlap
print(f"Dropping [{n_small} ({round(n_small/len(grid)*100,2)}%)] grids with < 10% fractional overlap.")
grid = grid[grid['overlap'] >= 0.10] # remove these observations


There are [57232] grids across [100] fires.


Index(['grid_index', 'grid_area', 'afd_count', 'unique_days', 'overlap',
       'frp_csum', 'frp_max', 'frp_min', 'frp_mean', 'frp_p90', 'frp_p95',
       'frp_p97', 'frp_p99', 'frp_first', 'day_max_frp', 'dt_max_frp',
       'first_obs_date', 'last_obs_date', 't4_max', 't4_mean', 't5_max',
       't5_mean', 'day_count', 'night_count', 'frp_max_day', 'frp_max_night',
       'frp_csum_day', 'frp_csum_night', 'frp_mean_day', 'frp_mean_night',
       'frp_p90_day', 'frp_p90_night', 'frp_p95_day', 'frp_p95_night',
       'frp_p97_day', 'frp_p97_night', 'frp_p99_day', 'frp_p99_night',
       'frp_first_day', 'frp_first_night', 'Fire_ID', 'Fire_Name', 'geometry',
       'x', 'y'],
      dtype='object')

Dropping [1395] duplicate grids.

Fractional overlap:
count    56767.000000
mean         2.029756
std          2.004417
min          0.000003
25%          0.591083
50%          1.498852
75%          2.779104
max         24.470760
Name: overlap, d

In [20]:
# Load the FORTYPCD and the Tree Table summaries

# FORTYPCD
fp = os.path.join(projdir,'data/tabular/mod/viirs_snpp_jpss1_gridstats_fortypnm_gp.csv')
fortyp = pd.read_csv(fp)
print(f"FORTYPCD columns:\n{fortyp.columns}\n")

# Tree Table
fp = os.path.join(projdir,'data/tabular/mod/gridstats_treetable.csv')
trees = pd.read_csv(fp)
trees.drop(columns=['Unnamed: 0','forest_pct'], inplace=True)
print(f"Tree Table columns:\n{trees.columns}\n")

# Check on how many grids match
print(f"\tFORTYP unique grids: {len(fortyp['grid_index'].unique())}")
print(f"\t\t total rows: {len(fortyp)}")
print(f"\tTree Table unique grids: {len(trees['grid_index'].unique())}")
print(f"\t\t total rows: {len(trees)}")

FORTYPCD columns:
Index(['Unnamed: 0', 'grid_index', 'fortypnm_gp', 'count', 'total_pixels',
       'pct_cover', 'forest_pixels', 'forest_pct'],
      dtype='object')

Tree Table columns:
Index(['grid_index', 'species_gp_n', 'tmid_n', 'ba_live', 'ba_dead', 'ba_ld',
       'tpp_live', 'tpp_dead', 'tpp_ld', 'qmd_live', 'qmd_dead', 'qmd_ld',
       'tree_ht_live', 'tree_ht_dead', 'balive_total', 'badead_total',
       'ba_ld_total', 'tpp_live_total', 'tpp_dead_total', 'tpp_ld_total',
       'qmd_live_total', 'qmd_dead_total', 'qmd_ld_total', 'ba_live_pr',
       'ba_dead_pr', 'ba_ld_pr', 'tpp_live_pr', 'tpp_dead_pr', 'tpp_ld_pr',
       'qmd_live_pr', 'qmd_dead_pr', 'qmd_ld_pr'],
      dtype='object')

	FORTYP unique grids: 53941
		 total rows: 185629
	Tree Table unique grids: 53947
		 total rows: 259997


In [21]:
# check how many matching grids
match = fortyp[fortyp['grid_index'].isin(trees['grid_index'].unique())]
# Check on how many grids match
print(f"\tFORTYP unique grids: {len(match['grid_index'].unique())}")
print(f"\t\t total rows: {len(match)}")
print(f"\tTree Table unique grids: {len(trees['grid_index'].unique())}")
print(f"\t\t total rows: {len(trees)}")
del match
gc.collect()

	FORTYP unique grids: 53886
		 total rows: 185494
	Tree Table unique grids: 53947
		 total rows: 259997


0

In [22]:
# calculate the dominant forest type using the FORTYPCD
dfortyp = fortyp.loc[fortyp.groupby('grid_index')['pct_cover'].idxmax()]
dfortyp = dfortyp[['grid_index','fortypnm_gp','pct_cover','forest_pixels','forest_pct']]
dfortyp = dfortyp[dfortyp['grid_index'].isin(trees['grid_index'].unique())] # retain matching grids
dfortyp.rename(columns={'pct_cover': 'fortyp_pct'}, inplace=True)
dfortyp.columns

Index(['grid_index', 'fortypnm_gp', 'fortyp_pct', 'forest_pixels',
       'forest_pct'],
      dtype='object')

In [23]:
dfortyp['fortypnm_gp'].unique()

array(['Piñon-juniper', 'Ponderosa', 'Quaking aspen', 'Mixed-conifer',
       'Lodgepole', 'Spruce-fir'], dtype=object)

In [24]:
# Subset the FORTYP metrics to retain those which match our FORTYPCDthe top species
spps = ['Ponderosa', 'Lodgepole', 'Spruce-fir', 'Quaking aspen', 'Mixed-conifer', 'Piñon-juniper']
dfortyp_sp = dfortyp[dfortyp['fortypnm_gp'].isin(spps)]
print(f"{len(dfortyp_sp)} / {len(dfortyp)} with our predominant species.")
# del dfortyp
# gc.collect()

53886 / 53886 with our predominant species.


In [25]:
# Check on how many matches there are between the dominant type and the Tree Table
matches = (dfortyp_sp['grid_index'].isin(trees[trees['species_gp_n'].isin(dfortyp_sp['fortypnm_gp'])]['grid_index'])).sum()
# Print the results
total_grids = dfortyp_sp['grid_index'].nunique()
print(f"\nTotal grids: {total_grids}\n")
print(f"{matches} ({(matches / total_grids) * 100:.2f}%)\n")
del matches, total_grids
gc.collect()


Total grids: 53886

53886 (100.00%)



0

In [26]:
# merge to the tree table metrics
tree_metrics = dfortyp_sp.merge(trees, on='grid_index', how='left')
tree_metrics.head()

,grid_index,fortypnm_gp,fortyp_pct,forest_pixels,forest_pct,species_gp_n,tmid_n,ba_live,ba_dead,ba_ld,...,qmd_ld_total,ba_live_pr,ba_dead_pr,ba_ld_pr,tpp_live_pr,tpp_dead_pr,tpp_ld_pr,qmd_live_pr,qmd_dead_pr,qmd_ld_pr
0,34602,Piñon-juniper,81.767956,182,99.450549,Mixed-conifer,1,1757.987391,131.985888,1889.973279,...,792.238607,0.057034,0.037533,0.055037,0.058037,0.004221,0.058037,0.183416,0.546625,0.183416
1,34602,Piñon-juniper,81.767956,182,99.450549,Piñon-juniper,12,25271.768073,3234.725392,28506.493465,...,792.238607,0.819891,0.919860,0.830128,0.880543,0.991557,0.880543,0.256595,0.162172,0.256595
2,34602,Piñon-juniper,81.767956,182,99.450549,Ponderosa,4,3753.173956,149.829507,3903.003463,...,792.238607,0.121764,0.042607,0.113658,0.058519,0.004221,0.058519,0.408269,0.291202,0.408269
3,34602,Piñon-juniper,81.767956,182,99.450549,Quaking aspen,1,40.406737,0.000000,40.406737,...,792.238607,0.001311,0.000000,0.001177,0.002902,0.000000,0.002902,0.151720,0.000000,0.151720
4,34603,Piñon-juniper,56.886228,169,98.816568,Mixed-conifer,4,3991.778414,195.938694,4187.717108,...,612.193167,0.121807,0.054726,0.115200,0.114194,0.026909,0.114194,0.274633,0.475397,0.274633


In [27]:
print(len(tree_metrics))

259909


In [28]:
tree_metrics.columns

Index(['grid_index', 'fortypnm_gp', 'fortyp_pct', 'forest_pixels',
       'forest_pct', 'species_gp_n', 'tmid_n', 'ba_live', 'ba_dead', 'ba_ld',
       'tpp_live', 'tpp_dead', 'tpp_ld', 'qmd_live', 'qmd_dead', 'qmd_ld',
       'tree_ht_live', 'tree_ht_dead', 'balive_total', 'badead_total',
       'ba_ld_total', 'tpp_live_total', 'tpp_dead_total', 'tpp_ld_total',
       'qmd_live_total', 'qmd_dead_total', 'qmd_ld_total', 'ba_live_pr',
       'ba_dead_pr', 'ba_ld_pr', 'tpp_live_pr', 'tpp_dead_pr', 'tpp_ld_pr',
       'qmd_live_pr', 'qmd_dead_pr', 'qmd_ld_pr'],
      dtype='object')

In [29]:
tree_metrics[['grid_index','fortypnm_gp','fortyp_pct','species_gp_n',
              'tpp_ld_pr','ba_ld_pr','qmd_ld_pr']].head(12)

,grid_index,fortypnm_gp,fortyp_pct,species_gp_n,tpp_ld_pr,ba_ld_pr,qmd_ld_pr
0,34602,Piñon-juniper,81.767956,Mixed-conifer,0.058037,0.055037,0.183416
1,34602,Piñon-juniper,81.767956,Piñon-juniper,0.880543,0.830128,0.256595
2,34602,Piñon-juniper,81.767956,Ponderosa,0.058519,0.113658,0.408269
3,34602,Piñon-juniper,81.767956,Quaking aspen,0.002902,0.001177,0.151720
4,34603,Piñon-juniper,56.886228,Mixed-conifer,0.114194,0.115200,0.274633
5,34603,Piñon-juniper,56.886228,Piñon-juniper,0.665630,0.537862,0.221870
6,34603,Piñon-juniper,56.886228,Ponderosa,0.218051,0.345827,0.343040
7,34603,Piñon-juniper,56.886228,Quaking aspen,0.002125,0.001112,0.160457
8,34604,Piñon-juniper,52.095808,Mixed-conifer,0.068728,0.046173,0.172977
9,34604,Piñon-juniper,52.095808,Piñon-juniper,0.724584,0.666044,0.262333


In [30]:
# Load the climate and topography information

# topography
fp = os.path.join(projdir,'data/earth-engine/exports/gridstats_topo.csv')
topo = pd.read_csv(fp)
print(f"\n{topo.columns}\n")

# climate (gridmet)
fp = os.path.join(projdir,'data/earth-engine/exports/gridstats_gridmet.csv')
climate = pd.read_csv(fp)
print(f"\n{climate.columns}\n")


Index(['system:index', 'chili', 'elev', 'grid_index', 'slope', 'tpi', '.geo'], dtype='object')


Index(['system:index', 'Fire_ID', 'erc', 'erc_dv', 'first_obs', 'vpd',
       'vpd_dv', '.geo'],
      dtype='object')



In [31]:
# tidy and merge the climate/topo
climate.rename(columns={'first_obs': 'first_obs_date'}, inplace=True) # for joining to the grid data
climate = climate[['Fire_ID', 'first_obs_date', 'erc', 'erc_dv', 'vpd', 'vpd_dv']] # keep needed columns
climate['Fire_ID'] = climate['Fire_ID'].astype(str) # to match the grid column
grid['first_obs_date'] = grid['first_obs_date'].astype(str) # to match gee output
# merge climate to the grid by fire id and first acquisition day
grid_clim = grid.merge(climate, on=['Fire_ID','first_obs_date'], how='left')
print(f"\n{grid_clim.columns}\n")


Index(['grid_index', 'grid_area', 'afd_count', 'unique_days', 'overlap',
       'frp_csum', 'frp_max', 'frp_min', 'frp_mean', 'frp_p90', 'frp_p95',
       'frp_p97', 'frp_p99', 'frp_first', 'day_max_frp', 'dt_max_frp',
       'first_obs_date', 'last_obs_date', 't4_max', 't4_mean', 't5_max',
       't5_mean', 'day_count', 'night_count', 'frp_max_day', 'frp_max_night',
       'frp_csum_day', 'frp_csum_night', 'frp_mean_day', 'frp_mean_night',
       'frp_p90_day', 'frp_p90_night', 'frp_p95_day', 'frp_p95_night',
       'frp_p97_day', 'frp_p97_night', 'frp_p99_day', 'frp_p99_night',
       'frp_first_day', 'frp_first_night', 'Fire_ID', 'Fire_Name', 'geometry',
       'x', 'y', 'erc', 'erc_dv', 'vpd', 'vpd_dv'],
      dtype='object')



In [32]:
# merge the topography to the grid
topo = topo[['grid_index', 'elev', 'slope', 'chili', 'tpi']]
grid_clim_topo = grid_clim.merge(topo, on='grid_index', how='left')
grid_clim_topo.columns

Index(['grid_index', 'grid_area', 'afd_count', 'unique_days', 'overlap',
       'frp_csum', 'frp_max', 'frp_min', 'frp_mean', 'frp_p90', 'frp_p95',
       'frp_p97', 'frp_p99', 'frp_first', 'day_max_frp', 'dt_max_frp',
       'first_obs_date', 'last_obs_date', 't4_max', 't4_mean', 't5_max',
       't5_mean', 'day_count', 'night_count', 'frp_max_day', 'frp_max_night',
       'frp_csum_day', 'frp_csum_night', 'frp_mean_day', 'frp_mean_night',
       'frp_p90_day', 'frp_p90_night', 'frp_p95_day', 'frp_p95_night',
       'frp_p97_day', 'frp_p97_night', 'frp_p99_day', 'frp_p99_night',
       'frp_first_day', 'frp_first_night', 'Fire_ID', 'Fire_Name', 'geometry',
       'x', 'y', 'erc', 'erc_dv', 'vpd', 'vpd_dv', 'elev', 'slope', 'chili',
       'tpi'],
      dtype='object')

In [33]:
# subset columns to keep for modeling
grid_clim_topo = grid_clim_topo[['grid_index','Fire_ID','first_obs_date','frp_csum','frp_max','frp_max_day',
                                 'frp_max_night','frp_csum_day','frp_csum_night','afd_count','day_count','night_count',
                                 'erc','erc_dv','vpd','vpd_dv','elev','slope','chili','tpi','x','y']]
tree_metrics = tree_metrics[['grid_index','fortypnm_gp','fortyp_pct','forest_pct','species_gp_n',
                             'ba_live','ba_dead','ba_ld',
                             'tpp_live','tpp_dead','tpp_ld',
                             'qmd_live','qmd_dead','qmd_ld',
                             'ba_live_pr','ba_dead_pr','ba_ld_pr',
                             'tpp_live_pr','tpp_dead_pr','tpp_ld_pr',
                             'qmd_live_pr','qmd_dead_pr','qmd_ld_pr',
                             'tree_ht_live','tree_ht_dead']]
# merge the FRP, climate, and topogrpahy to the forest metrics table
grid_tm = tree_metrics.merge(grid_clim_topo, on='grid_index', how='inner')
print(grid_tm.head(3))

   grid_index    fortypnm_gp  fortyp_pct  forest_pct   species_gp_n  \
0       34602  Piñon-juniper   81.767956   99.450549  Mixed-conifer   
1       34602  Piñon-juniper   81.767956   99.450549  Piñon-juniper   
2       34602  Piñon-juniper   81.767956   99.450549      Ponderosa   

        ba_live      ba_dead         ba_ld    tpp_live   tpp_dead  ...  \
0   1757.987391   131.985888   1889.973279   26.767667   0.222214  ...   
1  25271.768073  3234.725392  28506.493465  406.122778  52.196950  ...   
2   3753.173956   149.829507   3903.003463   26.989881   0.222214  ...   

         erc     erc_dv       vpd    vpd_dv         elev     slope  \
0  67.272727  14.479287  1.125455  0.352746  2330.355876  8.706296   
1  67.272727  14.479287  1.125455  0.352746  2330.355876  8.706296   
2  67.272727  14.479287  1.125455  0.352746  2330.355876  8.706296   

        chili        tpi          x          y  
0  204.949908  22.736575 -109.55098  37.761913  
1  204.949908  22.736575 -109.55098  37

In [34]:
len(grid_tm)

255706

In [35]:
# save this file out.
out_fp = os.path.join(projdir,'data/tabular/mod/gridstats_fortypnm_gp_tm_ct.csv')
grid_tm.to_csv(out_fp)
print(f"Saved file to: {out_fp}")

Saved file to: /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/tabular/mod/gridstats_fortypnm_gp_tm_ct.csv
